# Phase 3: Microservice Identification (Grouping by Similar Services)

In [ ]:
version = "v_imen" # All options: v_imen, v_team
system = "pos" # All options: jforum, cargotracker, petclinic, pos
model_type = "codebert" # All options: ft_codebert, word2vec, albert, codebert, roberta, bert
best_community_detection_algorithm = 'Louvain' # All options: Louvain, EdMot, Infomap, LabelPropagation, FastGreedy, GirvanNewman

## 1.1 Create service graph

In [ ]:
import pandas as pd
import numpy as np
from scipy import spatial

In [ ]:
# Load the data
communities_df = pd.read_csv(f"generated_data/community/{version}_{system}_{best_community_detection_algorithm}_communities.csv")
class_graph_df = pd.read_csv(f"generated_data/graph/class/{version}_{system}_class_graph.csv")
embeddings_df = pd.read_csv(f"generated_data/embedding/{version}_{system}_{model_type}_embeddings.csv")

# Extract class names and their embeddings
class_names = embeddings_df.iloc[:, 0].str.split(';', expand=True)[0]
embeddings = embeddings_df.iloc[:, 1:].values
class_embeddings_dict = dict(zip(class_names, embeddings))

def compute_static_distances_for_service_pairs(class_graph, communities):
    merged_df = class_graph.merge(communities, left_on='class1', right_on='class_name').merge(
        communities, left_on='class2', right_on='class_name', suffixes=('_1', '_2')
    )
    inter_service_df = merged_df[merged_df['service_1'] != merged_df['service_2']]
    return inter_service_df.groupby(['service_1', 'service_2'])['static_distance'].sum().to_dict()

def compute_service_embeddings(embeddings_dict, communities):
    """Compute service embeddings by averaging class embeddings for each service, skipping missing embeddings."""
    service_embeddings = {}
    for service, class_group in communities.groupby('service')['class_name']:
        class_embeddings = [embeddings_dict[class_name] for class_name in class_group if class_name in embeddings_dict]
        if class_embeddings:
            service_embeddings[service] = np.mean(class_embeddings, axis=0)
    return service_embeddings

def compute_semantic_distances_for_service_pairs(embeddings_dict, communities):
    service_embeddings = compute_service_embeddings(embeddings_dict, communities)
    
    services = list(service_embeddings.keys())
    semantic_distances = {}
    for i, s1 in enumerate(services):
        for j, s2 in enumerate(services):
            if i != j:
                distance = 1 - spatial.distance.cosine(service_embeddings[s1], service_embeddings[s2])
                semantic_distances[(s1, s2)] = distance
    
    return semantic_distances

def normalize_data(data):
    min_val, max_val = min(data.values()), max(data.values())
    range_val = max_val - min_val
    return {k: (v - min_val) / range_val for k, v in data.items()} if range_val else {k: 0 for k, v in data.items()}

# Compute static and semantic distances
static_distances = compute_static_distances_for_service_pairs(class_graph_df, communities_df)
semantic_distances = compute_semantic_distances_for_service_pairs(class_embeddings_dict, communities_df)

# Normalize the distances
normalized_static_distances = normalize_data(static_distances)
normalized_semantic_distances = normalize_data(semantic_distances)

# Create the service graph DataFrame
service_graph_data = [
    [s1, s2, normalized_static_distances.get((s1, s2), 0), normalized_semantic_distances.get((s1, s2), 0)]
    for s1, s2 in static_distances.keys()
]
service_graph_df = pd.DataFrame(service_graph_data, columns=['service1', 'service2', 'static_distance', 'semantic_distance'])

# Save the DataFrame
filename = f"generated_data/graph/service/{version}_{system}_service_graph.csv"
service_graph_df.to_csv(filename, index=False)

## 1.2 Cluster services

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.manifold import MDS
import skfuzzy as fuzz
from utils import save_microservices_to_file, save_clusters_to_file

In [ ]:
def compute_edge_weight(semantic, static, alpha=0.5):
    if not (0 <= semantic <= 1) or not (0 <= static <= 1):
        raise ValueError("Both 'semantic' and 'static' values should be between 0 and 1.")
    
    beta = 1 - alpha
    return alpha * static + beta * semantic

In [ ]:
# Usage
services_graph = nx.DiGraph([
    (row['service1'], row['service2'], {"weight": compute_edge_weight(row['semantic_distance'], row['static_distance'], 1)}) # adjust alpha (weight of static distance)
    for _, row in service_graph_df.iterrows()
])

# Remove edges with weight 0
services_graph.remove_edges_from([(u, v) for u, v, weight in services_graph.edges(data='weight') if weight == 0])

# Retrieve the list of nodes
nodes_list = list(services_graph.nodes)

# Print pairs of nodes and their edge weights
# print("Edge weights:")
# for u, v, weight in services_graph.edges(data='weight'):
#     print(f"{u} - {v}: {weight}")

# Initialize a dictionary to hold pairwise distances with a default distance (indicating maximum dissimilarity)
default_distance = 1
distances = {node: {node2: default_distance for node2 in nodes_list} for node in nodes_list}

# Fill the dictionary with distances, taking into account actual edge weights and non-edges
for u, v, data in services_graph.edges(data=True):
    # In this context, we can consider distance as the inverse of weight, or as a dissimilarity measure.
    distances[u][v] = distances[v][u] = 1 - data['weight']

# Set the distance from a node to itself to be 0
for node in nodes_list:
    distances[node][node] = 0

# Convert the distances dictionary to a square matrix form
distance_matrix = pd.DataFrame(distances).values

#######################

# Compute shortest paths
shortest_distances = nx.floyd_warshall_numpy(services_graph, weight='weight')
# Replace inf values with 1e9
shortest_distances[shortest_distances == np.inf] = 1e9

# print("Shortest distances matrix:")
# print(shortest_distances)

# If you want to find the shortest distance between two specific nodes, you can do the following:
node_a = 'Application Service 5'
node_b = 'Entity Service 3'
node_a_index = nodes_list.index(node_a)
node_b_index = nodes_list.index(node_b)
distance = shortest_distances[node_a_index][node_b_index]
print(f"Shortest distance from {node_a} to {node_b}: {distance}")

### 1.2.1 Fuzzy C-means from Scikit

In [ ]:
# Constants (change if needed)
N_COMPONENTS = 2
DISSIMILARITY = 'precomputed'
FUZZINESS = 2
ERROR_THRESHOLD = 0.005
MAX_ITERATIONS = 1000
CLUSTER_RANGE = range(1, 100)

In [ ]:
def compute_2D_embedding(distances):
    """Computes a 2D embedding from the given distance matrix."""
    embedding = MDS(n_components=N_COMPONENTS, dissimilarity=DISSIMILARITY)
    return embedding.fit_transform(distances)

def determine_optimal_clusters(data):
    """Determines the optimal number of clusters using the Elbow method."""
    fpc_values = []
    for c_value in CLUSTER_RANGE:
        _, _, _, _, _, _, fpc = fuzz.cmeans(
            data.T, 
            c=c_value, 
            m=FUZZINESS, 
            error=ERROR_THRESHOLD, 
            maxiter=MAX_ITERATIONS
        )
        fpc_values.append(fpc)

    plt.figure()
    plt.plot(CLUSTER_RANGE, fpc_values)
    plt.title('Fuzzy Partition Coefficient (FPC) for different cluster numbers')
    plt.xlabel('Number of clusters')
    plt.ylabel('FPC')
    plt.grid(True)
    plt.show()

    return detect_elbow(fpc_values)

def detect_elbow(y_values):
    """Detects the 'elbow' in a list of y-values."""
    # Get coordinates of all the points
    n_points = len(y_values)
    all_coords = np.vstack((range(n_points), y_values)).T
    # Get vectors between all points from the first point to the last point
    first_point = all_coords[0]
    line_vector = all_coords[-1] - all_coords[0]
    line_vector_norm = line_vector / np.sqrt(np.sum(line_vector**2))
    
    # Get orthogonal vectors from the first point to all points
    vec_from_first = all_coords - first_point
    scalar_prod = np.sum(vec_from_first * line_vector_norm, axis=1)
    vec_from_first_parallel = np.outer(scalar_prod, line_vector_norm)
    vec_to_line = vec_from_first - vec_from_first_parallel
    
    # Compute the distance to the line
    dist_to_line = np.sqrt(np.sum(vec_to_line ** 2, axis=1))
    
    # Return the index of the point with max distance to the line
    idx_elbow = np.argmax(dist_to_line)
    return idx_elbow

def fuzzy_cmeans_clustering(data, optimal_clusters):
    """Clusters the data using Fuzzy C-Means."""
    cntr, u, _, _, _, _, _ = fuzz.cmeans(
        data.T, 
        c=optimal_clusters, 
        m=FUZZINESS, 
        error=ERROR_THRESHOLD, 
        maxiter=MAX_ITERATIONS
    )
    return np.argmax(u, axis=0)

def visualize_clusters(data, labels):
    """Visualizes the clustered data."""
    plt.figure(figsize=(10, 8))
    for cluster_num in np.unique(labels):
        cluster_points = data[labels == cluster_num]
        plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label=f'Microservice {cluster_num + 1}')

    plt.title('Fuzzy C-Means Clustering')
    plt.xlabel('MDS Dimension 1')
    plt.ylabel('MDS Dimension 2')
    plt.legend()
    plt.show()

In [ ]:
# transformed_data = compute_2D_embedding(shortest_distances)
optimal_clusters = determine_optimal_clusters(shortest_distances)
print('Optimal number of clusters: ', optimal_clusters)
labels = fuzzy_cmeans_clustering(shortest_distances, optimal_clusters)

# embedding_2d = MDS(n_components=N_COMPONENTS, dissimilarity=DISSIMILARITY).fit_transform(shortest_distances)
# visualize_clusters(embedding_2d, labels)

print(labels)

# Save results
save_microservices_to_file(labels, services_graph, communities_df, f"results/{version}_{system}_microservices.txt")

### 1.2.1 Custom fuzzy C-means

In [ ]:
class FuzzyCMeans:
    def __init__(self, m, membership_threshold, merge_threshold):
        """
        Initialize the FuzzyCMeans object with the specified parameters.
        
        Parameters:
        m (float): The fuzziness coefficient that determines the level of cluster fuzziness.
        membership_threshold (float): The proportion of the highest membership value 
                                      used as a cutoff for assigning services to additional clusters.
        merge_threshold (float): The threshold for determining when to merge two clusters
                                 based on the overlap of their members.
        """
        self.m = m
        self.membership_threshold = membership_threshold
        self.merge_threshold = merge_threshold
        self.coef = 2 / (m - 1)  # Precomputed coefficient for membership calculation.

    def calculate_membership(self, service_idx, center_idx, distances):
        """
        Calculate the fuzzy membership of a service to a cluster center.
        
        Parameters:
        service_idx (int): Index of the service for which membership is being calculated.
        center_idx (int): Index of the cluster center to which membership is being calculated.
        distances (list): Matrix of distances between services and cluster centers.
        
        Returns:
        float: The membership value of the service to the cluster center.
        """

        # Use the minimum distance as the effective distance to the center
        distance_to_center = distances[service_idx][center_idx]
        distance_from_center = distances[center_idx][service_idx]
        effective_distance = min(distance_to_center, distance_from_center)

        # This will hold the sum of all (effective_distance / other_effective_distance) ^ (2 / (m-1))
        # where other_effective_distance is the effective distance to other centers.
        membership_sum = 0.0

        if effective_distance == 0:
            return 1.0

        # Calculate the sum part of the membership function
        for other_center_idx in self.center_type_indices.values():
            if other_center_idx != service_idx:
                # Direct distances to other centers
                other_distance_to_center = distances[service_idx][other_center_idx]
                other_distance_from_center = distances[other_center_idx][service_idx]

                # Use the minimum distance as the other effective distance
                other_effective_distance = min(other_distance_to_center, other_distance_from_center)

                # The ratio is now (effective_distance / other_effective_distance)
                # We want a smaller number when the effective distance is smaller,
                # and a larger number when the other effective distance is larger.
                membership_ratio = effective_distance / (other_effective_distance + 1e-12)
                membership_sum += membership_ratio ** self.coef

        # The membership value is the inverse of the sum, normalized by the number of centers
        return 1 / membership_sum if membership_sum > 0 else 0

        
    def cluster_services(self, services_list, distances, service_type):
        """
        Cluster services based on their membership values to each cluster center.
        
        Parameters:
        services_list (list): List of all services to be clustered.
        distances (list): Matrix of distances between services and cluster centers.
        service_type (str): The type of service that defines the cluster centers.
        
        Returns:
        dict: A dictionary with cluster centers as keys and lists of services as values.
        """
        # Step 1: Initialize clusters and identify services that are centers based on the service type.
        self.center_type_indices = {
            service: idx for idx, service in enumerate(services_list) if service.startswith(service_type)
        }

        # Step 2: Calculate raw membership values for each service to each cluster center.
        raw_memberships = {
            service: [] for service in services_list
        }
        for idx, service in enumerate(services_list):
            for center_service, center_idx in self.center_type_indices.items():
                membership_value = self.calculate_membership(idx, center_idx, distances)
                raw_memberships[service].append((center_service, membership_value))

        # Step 3: Normalize membership values so that they sum to 1 for each service.
        normalized_memberships = {}
        for service, memberships in raw_memberships.items():
            total_membership = sum(membership for _, membership in memberships)
            normalized_memberships[service] = [
                (center_service, membership / total_membership if total_membership else 0)
                for center_service, membership in memberships
            ]

        # Print the normalized memberships for each service.
        # for service, memberships in normalized_memberships.items():
        #     print(f"{service}: {memberships}")

        # Step 4: Assign services to clusters based on normalized membership values.
        clusters = {center_service: [] for center_service in self.center_type_indices}
        for service, memberships in normalized_memberships.items():
            # Determine the highest normalized membership value.
            highest_membership_value = max(memberships, key=lambda x: x[1])[1]
            # Determine the primary cluster for this service.
            primary_cluster = max(memberships, key=lambda x: x[1])[0]
            clusters[primary_cluster].append(service)
            
            # Assign service to other clusters where the membership value is above the relative threshold.
            relative_threshold = highest_membership_value * self.membership_threshold
            for center_service, membership_value in memberships:
                if membership_value >= relative_threshold and center_service != primary_cluster:
                    clusters[center_service].append(service)

        # Step 5: Merge clusters if they have a significant overlap as defined by merge_threshold.
        clusters = self.merge_clusters_based_on_overlap(clusters)

        # Step 6: Remove duplicate services from each cluster to maintain unique membership.
        for center_service, members in clusters.items():
            clusters[center_service] = list(set(members))

        return clusters

    def calculate_overlap(self, cluster_a, cluster_b):
        """
        Calculate the overlap between two clusters.
        
        Parameters:
        cluster_a (list): The first cluster to compare.
        cluster_b (list): The second cluster to compare.
        
        Returns:
        float: The ratio of the intersection to the union of the two clusters.
        """
        intersection = set(cluster_a).intersection(cluster_b)
        union = set(cluster_a).union(cluster_b)
        return len(intersection) / len(union) if union else 0

    def merge_clusters_based_on_overlap(self, clusters):
        """
        Merge clusters based on the overlap between their services.
        
        Parameters:
        clusters (dict): Dictionary with cluster centers as keys and lists of services as values.
        
        Returns:
        dict: The updated dictionary of clusters after merging.
        """
        # Find all pairs of clusters that have an overlap above the merge threshold.
        clusters_to_merge = []
        keys = list(clusters.keys())
        for i, key_i in enumerate(keys):
            for j in range(i + 1, len(keys)):
                key_j = keys[j]
                if self.calculate_overlap(clusters[key_i], clusters[key_j]) > self.merge_threshold:
                    clusters_to_merge.append((key_i, key_j))

        # Merge the identified clusters.
        for key_i, key_j in clusters_to_merge:
            if key_i in clusters and key_j in clusters:
                clusters[key_i].extend(clusters[key_j])
                clusters[key_i] = list(set(clusters[key_i]))
                del clusters[key_j]

        return clusters

In [ ]:
# Constants (change if needed)
FUZZINESS = 2
MEMBERSHIP_THRESHOLD = 0.8
MERGE_THRESHOLD = 0.5
CENTER_TYPE = 'Entity'

In [ ]:
# Usage
fuzzy_c_means = FuzzyCMeans(m=FUZZINESS, membership_threshold=MEMBERSHIP_THRESHOLD, merge_threshold=MERGE_THRESHOLD) # change thresholds

# Perform clustering with the best thresholds
clusters = fuzzy_c_means.cluster_services(nodes_list, distance_matrix, service_type=CENTER_TYPE) # change service type

# Output clusters
for center, members in clusters.items():
    print(f"Cluster centered at {center}: {members}")

# Save results
save_clusters_to_file(clusters, communities_df, f"results/{version}_{system}_microservices.txt")

### 1.2.2 Graph clustering

In [ ]:
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform

In [ ]:
# Perform the hierarchical clustering
Z = linkage(squareform(distance_matrix), method='average')

# Now we can form flat clusters from the hierarchical clustering defined by the linkage matrix
# We choose a max_d that cuts the dendrogram to form clusters as desired. 
max_d = 0.95
clusters = fcluster(Z, max_d, criterion='distance')

# Map the flat clusters back to our nodes
cluster_map = {node: cluster for node, cluster in zip(nodes_list, clusters)}

cluster_map_values = np.array(list(cluster_map.values()))

# Save results
save_microservices_to_file(cluster_map_values - 1, services_graph, communities_df, f"results/{version}_{system}_microservices.txt")

cluster_map